# 第8章 对象引用、可变性和垃圾回收

## 8.1 变量不是盒子

In [75]:
a = [1, 2, 3]
b = a
a.append(4)
b

[1, 2, 3, 4]

## 8.2 标识、相等性和别名

In [76]:
charles = {'name': 'Charles L. Dodgson', 'born': 1832}
lewis = charles
lewis is charles , id(charles) == id(lewis)

(True, True)

In [77]:
lewis['balance'] = 950
charles

{'name': 'Charles L. Dodgson', 'born': 1832, 'balance': 950}

In [78]:
alex = {'name': 'Charles L. Dodgson', 'born': 1832, 'balance': 950}
alex == charles, alex is charles

(True, False)

### 8.2.1 在==和is之间选择

In [79]:
x, y = [1], [1]
x is None, x is not None, x is y, x == y, id(x) == id(y), x.__eq__(y)

(False, True, False, True, False, True)

### 8.2.2 元组的相对不可变性

In [80]:
t1 = (1, 2, [30, 40])
t2 = (1, 2, [30, 40])
t1 == t2

True

In [81]:
id(t1[-1])

2263072124288

In [82]:
t1[-1].append(99)
t1

(1, 2, [30, 40, 99])

In [83]:
id(t1[-1]), t1 == t2

(2263072124288, False)

## 8.3 默认做浅复制

In [84]:
l1 = [3, [55, 44], (7, 8, 9)]
l2 = list(l1)
l2, l2 == l1, l2 is l1

([3, [55, 44], (7, 8, 9)], True, False)

示例 8-8 校车乘客在途中上车和下车

In [85]:
class Bus:

    def __init__(self, passengers=None):
        self.passengers = [] if passengers is None else list(passengers)
    
    def pick(self, name):
        self.passengers.append(name)
    
    def drop(self, name):
        self.passengers.remove(name)
    
    def __repr__(self) -> str:
        return str(self.passengers)

示例 8-9 使用copy和deepcopy产生的影响

In [86]:
import copy
bus1 = Bus(['Alice', 'Bill', 'Claire', 'David'])
bus2 = copy.copy(bus1)
bus3 = copy.deepcopy(bus1)
id(bus1), id(bus2), id(bus3)

(2263061591504, 2263072770192, 2263073778960)

In [87]:
bus1.drop('Bill')
bus2

['Alice', 'Claire', 'David']

In [88]:
id(bus1.passengers), id(bus2.passengers), id(bus3.passengers)

(2263061582656, 2263061582656, 2263073978560)

示例 8-10 循环引用: b引用a, 然后追加到a中, deepcopy 会想办法复制a

In [89]:
a = [10, 20]
b = [a, 30]
a.append(b)
a

[10, 20, [[...], 30]]

In [90]:
from copy import deepcopy

c = deepcopy(a)
c

[10, 20, [[...], 30]]

## 8.4 函数的参数作为引用时

示例 8-11 函数可能会修改接收到的任何可变对象 

In [91]:
def f(a, b):
    a += b
    return a

x, y = 1, 2
f(x, y), x, y

(3, 1, 2)

In [92]:
a, b = [1, 2], [3, 4]   # 说明 a += b 对于列表而言, `__iadd__` 返回的是它自己
f(a, b), a, b

([1, 2, 3, 4], [1, 2, 3, 4], [3, 4])

In [93]:
t, u = (10, 20), (30, 40)
f(t, u), t, u

((10, 20, 30, 40), (10, 20), (30, 40))

### 8.4.1 不要使用可变类型作为参数的默认值

示例 8-12 一个简单的类，说明可变默认值的危险

In [94]:
class HauntedBus:
    """A bus model haunted by ghost passengers"""

    def __init__(self, passengers=[]):  # <1>
        self.passengers = passengers  # <2>

    def pick(self, name):
        self.passengers.append(name)  # <3>

    def drop(self, name):
        self.passengers.remove(name)

示例 8-13 备受幽灵乘客折磨的校车

In [95]:
bus1 = HauntedBus(['Alice', 'Bill'])
bus1.passengers

['Alice', 'Bill']

In [96]:
bus1.pick('Chartlie')
bus1.drop('Alice')
bus1.passengers

['Bill', 'Chartlie']

In [97]:
bus2 = HauntedBus()
bus2.pick('Carrie')
bus2.passengers

['Carrie']

In [98]:
bus3 = HauntedBus()
bus3.passengers

['Carrie']

In [99]:
bus3.pick('Dave')
bus2.passengers

['Carrie', 'Dave']

In [100]:
bus2.passengers is bus3.passengers

True

In [101]:
bus1.passengers

['Bill', 'Chartlie']

> 默认值通常在定义函数时计算(通常在加载模块时), 因此默认值变成了函数对象的属性。因此, 如果默认值是可变对象, 而且修改了它的值, 那么后续的函数调用都会受到影响

In [102]:
HauntedBus.__init__.__defaults__

(['Carrie', 'Dave'],)

### 8.4.2 防御可变参数

In [103]:
class TwilightBus:
    """A bus model that makes passengers vanish"""

    def __init__(self, passengers=None):
        if passengers is None:
            self.passengers = []  # <1>
        else:
            self.passengers = passengers  #<2>

    def pick(self, name):
        self.passengers.append(name)

    def drop(self, name):
        self.passengers.remove(name)  # <3>

示例 8-14 从TwilightBus 下车后, 乘客消失了

In [104]:
basketball_team = ['Sue', 'Tina', 'Maya', 'Diana', 'Pat']
bus = TwilightBus(basketball_team)
bus.drop('Tina')
bus.drop('Pat')
basketball_team

['Sue', 'Maya', 'Diana']

In [105]:
class TwilightBus:
    """A bus model that makes passengers vanish"""

    def __init__(self, passengers=None):
        if passengers is None:
            self.passengers = []  # <1>
        else:
            self.passengers = list(passengers)  #<2>

    def pick(self, name):
        self.passengers.append(name)

    def drop(self, name):
        self.passengers.remove(name)  # <3>

    def __repr__(self) -> str:
        return str(self.passengers)

In [106]:
basketball_team = ['Sue', 'Tina', 'Maya', 'Diana', 'Pat']
bus = TwilightBus(basketball_team)
bus.drop('Tina')
bus.drop('Pat')
basketball_team, bus

(['Sue', 'Tina', 'Maya', 'Diana', 'Pat'], ['Sue', 'Maya', 'Diana'])

## 8.5 del 和 垃圾回收

> 对象绝不会自行销毁; 然而， 无法得到对象时, 可能会被当作垃圾回收

示例 8-16 没有指向对象的引用时, 监视对象生命结束时的情形

In [107]:
import weakref
s1 = {1, 2, 3}
s2 = s1
bye  = lambda : print('Gone with the wind...')
ender = weakref.finalize(s1, bye)
ender.alive

True

In [108]:
del s1
ender.alive

True

In [109]:
s2 = 'spam'

Gone with the wind...


In [110]:
s2

'spam'

In [111]:
ender.alive

False

## 8.6 弱引用

> 弱引用不会增加对象的引用数量。 引用的目标对象称为所指对象。弱引用不会妨碍所指对象当作垃圾回收

示例 8-17 弱引用时可调用的对象, 返回的是被引用的对象; 如果所指对象不存在了, 返回None

In [9]:
import weakref

class MyClass:
    def __init__(self, value):
        self.value = value
# 创建一个对象
obj = MyClass(10)
# 创建一个指向 obj 的弱引用
weak_obj_ref = weakref.ref(obj)
# 通过弱引用访问原对象
print("Weak reference value:", weak_obj_ref().value)  # 输出 10
# 删除强引用
o2 = obj
del obj, o2
# 尝试通过弱引用访问对象
print("Weak reference after deleting obj:", weak_obj_ref())  # 输出 None


Weak reference value: 10
Weak reference after deleting obj: None


In [11]:
s = {1, 2, 3}
weak_set_ref = weakref.ref(s)
print("Weak reference value:", weak_set_ref())  # 输出 10
del s
print("Weak reference after deleting s:", weak_set_ref())  # 输出 None


Weak reference value: {1, 2, 3}
Weak reference after deleting s: None


## 8.7 Python对不可变类型施加的把戏

示例 8-20 使用另一个元组构建元组, 得到的其实是同一个元组

In [12]:
t1 = (1, 2, 3)
t2 = tuple(t1)
t1 is t2

True

In [13]:
t3 = t1[:]
t1 is t3

True

In [16]:
import copy
t4 = copy.copy(t1)
t5 = copy.deepcopy(t1)
t1 is t4, t1 is t5

(True, True)

In [18]:
t1 = (1, 2, [3, 4])
t2 = tuple(t1)
t3 = t1[:]
t4 = copy.deepcopy(t1)
t1 is t2, t1 is t3, t1 is t4

(True, True, False)

In [19]:
t1 , t2 = (1, 2, 4), (1, 2, 4)
t1 is t2

True

In [20]:
s1, s2 = "ABC", "ABC"
s1 is s2

True

> 共享字符串字面量是一种优化措施, 称为驻留

<mark/> 注意: 千万不要依赖字符串或元组等不可变序列的驻留！ 比较字符串或者整数相等时, 应该使用 == 而不是 is。 驻留是Python解释器内部使用的一个特性</mark>

## 8.8 本章小结

<img src="./images/第八章总结.jpg" width="70%">